In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import pymongo
from openai import AzureOpenAI, BadRequestError
from dotenv import load_dotenv
import re
import logging
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from mitigating_aggravating_ai_lists import aggravating_list, mitigating_list
from bson import ObjectId
import json
from config import db_config
import time

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
load_dotenv(os.path.abspath(os.path.join(os.getcwd(), "../config/.env")))

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=os.getenv("AZURE_OPENAI_VERSION")
)

In [5]:
# Hugging Face config
tokenizer = AutoTokenizer.from_pretrained(
    "NbAiLab/nb-bert-base-ner", model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(
    "NbAiLab/nb-bert-base-ner")

/Users/henrik.boe/repos/Github/smeinrich/dommedag/venv/lib/python3.11/site-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [6]:
COL_LOCAL_DOMMER = db_config.DB_LOCAL_DOMMEDAG["dommer"]

## Functions


In [7]:
# Imports judgements from the database or from a json file.
def import_judgements(from_file: bool = False, file_path: str = "../input/judgements.json") -> list | pymongo.cursor.Cursor:
    if from_file:
        with open(file_path, "r") as f:
            return json.load(f)
    else:
        return COL_LOCAL_DOMMER.find(no_cursor_timeout=True)


# Updates judgements in the database or json file.
def update_judgement(judgements: list, judgement_id: str, circumstance_types: list, mitigating_or_aggravating: str, to_file: bool = False, file_path: str = "../input/judgements.json"):
    if to_file:
        for judgement in judgements:
            if judgement["_id"] == judgement_id:
                judgement[mitigating_or_aggravating] = circumstance_types
                break
        with open(file_path, 'w') as file:
            json.dump(judgements, file, indent=4)
    else:
        COL_LOCAL_DOMMER.update_one(
            {'_id': ObjectId(judgement_id)},
            {'$push': {mitigating_or_aggravating: circumstance_types}}
        )


# Splits the string into paragraphs using two newlines, including whitespace between them, as a separator
def get_paragraphs(str):
    paragraphs = re.split(r"\n\s*\n", str)
    return paragraphs


def is_analyzed(collection):
  # Check if "skjerpende_formildende_sjekket" field exists and is True
  return collection.get("skjerpende_formildende_sjekket", False) == True



def get_ai_response(system_prompt, user_example_1, assistant_example_1, user_example_2, assistant_example_2, user_example_3, assistant_example_3, paragraph):
    completion = client.chat.completions.create(
        model="nrkddivundersokendesorost1106",
        response_format={ "type": "json_object" },
        messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_example_1
                },
                {
                    "role": "assistant",
                    "content": assistant_example_1
                },
                {
                    "role": "user",
                    "content": user_example_2
                },
                {
                    "role": "assistant",
                    "content": assistant_example_2
                },
                {
                    "role": "user",
                    "content": user_example_3
                },
                {
                    "role": "assistant",
                    "content": assistant_example_3
                },
                {
                    "role": "user",
                    "content": paragraph
                },
            ],
        temperature=0,
        max_tokens=2000,)
    
    response = completion.choices[0].message.content
    time.sleep(3)
    return json.loads(response)


# Replaces names in a string with only the text "Navn".
def replace_names_with_placeholder(input_str):
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    ner_results = nlp(input_str)
    reversed_ner_results = list(reversed(ner_results))

    for entity in reversed_ner_results:
        if entity["entity"] == "B-PER":
            if "##" in entity["word"]:
                input_str = input_str[:entity["start"]] + \
                    input_str[entity["end"]:]
            else:
                input_str = input_str[:entity["start"]] + "Navn" + input_str[entity["end"]:]
        elif entity["entity"] == "I-PER":
            if "##" in entity["word"]:
                input_str = input_str[:entity["start"]] + \
                    input_str[entity["end"]:]
            else:
                input_str = input_str[:entity["start"]] + input_str[entity["end"]:]

    return input_str.replace("  ", " ")



def replace_dates(text):
    # Pattern for "dd.mm.yyyy" format
    pattern1 = r"\d{1,2}\.\s?\d{1,2}\.\s?\d{4}"

    # Pattern for "dd. month yyyy" format
    months = "januar|februar|mars|april|mai|juni|juli|august|september|oktober|november|desember"
    pattern2 = r"\d{1,2}\.\s?(" + months + r")(\s?\d{4})?"

    # Combined pattern with an 'or' condition
    combined_pattern = f"({pattern1})|({pattern2})"

    return re.sub(combined_pattern, "DATO", text)


def count_word(text, word):
  text = text.lower()
  word = word.lower()
  words = text.split()
  count = words.count(word)

  return count

# Checks if mitigating or aggravating circumstances are written in past tense.
# Returns True if any of the past tense formulations in the list are found AND the word mitigating or aggravating is only found once in the paragraph
def circumstance_is_in_past_tense(paragraph, mitigating=True) -> bool:
    if mitigating:
        if count_word(paragraph, "formildende") == 1 or count_word(paragraph, "formildande") == 1:
            mitigating_past_tense_list = ["var formildende", "var formildande", "forelå formildende", "forelå formildande", "formildende omstendigheter var", "formildande omstende var"]
            for string in mitigating_past_tense_list:
                if string in paragraph:
                    logging.info(f"Found paragraph in past tense: {paragraph}")
                    return True
        return False
    else:
        paragraph = paragraph.replace("straffeskjerpende", "skjerpende")
        paragraph = paragraph.replace("straffeskjerpande", "skjerpande")
        if count_word(paragraph, "skjerpende") == 1 or count_word(paragraph, "skjerpande") == 1:
            aggravating_past_tense_list = ["var skjerpende", "var skjerpande", "forelå skjerpende", "forelå skjerpande", "skjerpende omstendigheter var", "skjerpande omstende var"]
            for string in aggravating_past_tense_list:
                if string in paragraph:
                    logging.info(f"Found paragraph in past tense: {paragraph}")
                    return True
        return False



## OpenAI prompts


In [8]:
system_prompt_aggravating = """
Du er en assistent som leser korte tekstutdrag fra dommer. Du skal avgjøre om teksten inneholder informasjon om hvilke eventuelle skjerpende eller straffeskjerpende omstendigheter det er i saken. Du skal identifisere de mest relevante skjerpende eller straffeskjerpende omstendighete fra listen nedenfor.
Du skal kun nevne omstendigheter det står at er skjerpende eller straffeskjerpende for denne dommen, ikke omstendigheter for tidligere dommer. Dersom det ikke står at omstendigheten er skjerpende eller straffeskjerpende, eller hvis teksten om omstendigheten er skrevet i fortid, skal du svare {"skjerpende": "false"}.
Du skal svare i json-format på denne måten: {"skjerpende": "drapstrusler"}. Dersom du finner flere skjerpende omstendigheter skiller du dem med komma på denne måten: {"skjerpende": "drapstrusler, trygdesvindel"}.
Hvis du ikke finner en relevant omstendighet i listen, svarer du {"skjerpende": "andre omstendigheter"}. Hvis det mangler informasjon om skjerpende eller straffeskjerpende omstendigheter, eller hvis du ikke klarer å tolke det, skal du svare {"skjerpende": "false"}.
Hvis formuleringen om omstendigheten er skrevet i fortid, for eksempel "det var skjerpende" eller "det forelå skjerpende", skal du svare {"skjerpende": "false"}.
Her er listen:
""" + aggravating_list

system_prompt_mitigating = """
Du er en assistent som leser korte tekstutdrag fra dommer. Du skal avgjøre om teksten inneholder informasjon om hvilke eventuelle formildende omstendigheter det er i saken. Du skal identifisere de mest relevante formildende omstendighetene fra listen nedenfor.
Du skal kun nevne omstendigheter det står at er formildende for denne saken, ikke omstendigheter som er formildende for tidligere rettssaker. Dersom det ikke står at omstendigheten er formildende, eller hvis teksten om omstendigheten er skrevet i fortid, skal du svare {"formildende": "false"}.
Du skal svare i json-format på denne måten: {"formildende": "selvforsvar"}. Dersom du finner flere formildende omstendigheter skiller du dem med komma på denne måten: {"formildende": "selvforsvar, tilståelse"}.
Hvis du ikke finner en relevant omstendighet i listen, svarer du {"formildende": "andre omstendigheter"}. Hvis det mangler informasjon om formildende omstendigheter, eller hvis du ikke klarer å tolke det, skal du svare {"formildende": "false"}.
Hvis formuleringen om omstendigheten er skrevet i fortid, for eksempel "det var formildende" eller "det forelå formildende", skal du svare {"formildende": "false"}.
Her er listen:
""" + mitigating_list

In [9]:
# Examples for aggravating paragraphs and assistant response

user_example_1_aggravating = "I skjerpende retning legger retten vekt på at Navn kjørte i ruspåvirket tilstand. Det er også skjerpende at vedkommende har utøvd psykisk vold mot fornærmede."

assistant_example_1_aggravating = """{"skjerpende": "kjøring under ruspåvirkning, psykisk vold"}"""

user_example_2_aggravating = """
Navn ble ved Kristiansand tingretts dom av DATO dømt til forvaring i 16 år 
med minstetid på 10 år for overlagt drap under særdeles skjerpende omstendigheter. Etter anke ble straffen 
fastsatt til forvaring.
"""

assistant_example_2_aggravating = """{"skjerpende": "false"}"""

user_example_3_aggravating = """
Overtredelsen i post I ville medført en kortere betinget straff, 
og tillegges vekt i straffeskjerpende retning.
"""

assistant_example_3_aggravating = """{"skjerpende": "andre omstendigheter"}"""

# Examples for mitigating paragraphs and assistant response

user_example_1_mitigating = "I formildende retning legger retten vekt på at Navn har gitt en uforbeholden tilståelse. Det er i tillegg formildende at saksbehandlingstiden har vært lang."

assistant_example_1_mitigating = """{"formildende": "tilståelse, lang saksbehandlingstid"}"""

user_example_2_mitigating = """
Saken gjaldt straffutmåling for uaktsomt heleri. Det var formildende at de domfelte 
hadde samarbeidet med politiet og møtt som vitner i saken mot hovedmannen, 
og at straffeforfølgningen hadde tatt lang tid.
"""

assistant_example_2_mitigating = """{"formildende": "false"}"""

user_example_3_mitigating = """
Det forelå formildende omstendigheter, ettersom hun hadde daglig omsorg for det yngste barnet. 
Dommen er etter gammel straffelov, og før strafferammen ble 
økt, men samtidig er tidsperioden lengre enn i denne sak.
"""

assistant_example_3_mitigating = """{"formildende": "false"}"""

## Main processing loop


In [10]:
# If importing from json file, set the first argument to True and optionally the second argument to file path ("If file path is different than "../input/judgements.json"")
judgements = import_judgements()

/Users/henrik.boe/repos/Github/smeinrich/dommedag/venv/lib/python3.11/site-packages/pymongo/collection.py:1650: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


In [11]:
num = 0
num_total_judgements = COL_LOCAL_DOMMER.count_documents({})

# Loop through judgements
for judgement in judgements:
    judgement_id = judgement["_id"]
    num += 1
    logging.info(f"Judgement {num} of {num_total_judgements}")
    logging.info(judgement_id)

    if not is_analyzed(judgement):
        judgement_text = judgement["dom_tekst"]
        # Split text into list of paragraphs
        paragraphs = get_paragraphs(judgement_text)

        aggravating_circumstances = []
        mitigating_circumstances = []

        for paragraph in paragraphs:
            try:
                if "skjerpende" in paragraph.lower() or "skjerpande" in paragraph.lower():

                    if circumstance_is_in_past_tense(paragraph, False):
                        continue

                    paragraph_without_names = replace_names_with_placeholder(
                        paragraph)
                    paragraph_without_names_and_dates = replace_dates(
                        paragraph_without_names)
                    ai_response = get_ai_response(system_prompt_aggravating, user_example_1_aggravating, assistant_example_1_aggravating, user_example_2_aggravating,
                                                assistant_example_2_aggravating, user_example_3_aggravating, assistant_example_3_aggravating, paragraph_without_names_and_dates)
                    if ai_response == {"skjerpende": "false"}:
                        logging.info("No match for aggravating circumstance:")
                        logging.info(paragraph_without_names_and_dates)
                    else:
                        logging.info(ai_response)
                        logging.info(paragraph_without_names_and_dates)
                        if "," in ai_response["skjerpende"]:
                            ai_response["skjerpende"] = [i.strip() for i in ai_response["skjerpende"].split(",")]
                            aggravating_circumstances.extend(ai_response["skjerpende"])
                        else:
                            aggravating_circumstances.append(
                                ai_response["skjerpende"])
                if "formildende" in paragraph.lower() or "formildande" in paragraph.lower():
                    if circumstance_is_in_past_tense(paragraph):
                        continue

                    paragraph_without_names = replace_names_with_placeholder(
                        paragraph)
                    paragraph_without_names_and_dates = replace_dates(
                        paragraph_without_names)
                    ai_response = get_ai_response(system_prompt_mitigating, user_example_1_mitigating, assistant_example_1_mitigating, user_example_2_mitigating,
                                                assistant_example_2_mitigating, user_example_3_aggravating, assistant_example_3_aggravating, paragraph_without_names_and_dates)
                    if ai_response == {"formildende": "false"}:
                        logging.info("No match for mitigating circumstance:")
                        logging.info(paragraph_without_names_and_dates)
                    else:
                        logging.info(ai_response)
                        logging.info(paragraph_without_names_and_dates)
                        if "," in ai_response["formildende"]:
                            ai_response["formildende"] = [i.strip() for i in ai_response["formildende"].split(",")]
                            mitigating_circumstances.extend(ai_response["formildende"])
                        else:
                            mitigating_circumstances.append(
                                ai_response["formildende"])
            except BadRequestError as e:
                logging.info(f"BadRequestError: {e}")

        aggravating_circumstances = list(set(aggravating_circumstances))
        mitigating_circumstances = list(set(mitigating_circumstances))

        logging.info(aggravating_circumstances)
        if aggravating_circumstances:
            update_judgement(judgements, judgement_id,
                            aggravating_circumstances, "skjerpende")

        logging.info(mitigating_circumstances)
        if mitigating_circumstances:
            update_judgement(judgements, judgement_id,
                            mitigating_circumstances, "formildende")
            

        COL_LOCAL_DOMMER.update_one(
                {'_id': ObjectId(judgement_id)},
                {'$set': {"skjerpende_formildende_sjekket": True}}
            )
    else:
        logging.info("Is already analyzed.")

2024-05-10 20:12:21,774 - INFO - Judgement 1 of 9019
2024-05-10 20:12:21,774 - INFO - 650c3a798fc6aa8c775aa20a
2024-05-10 20:12:21,775 - INFO - Is already analyzed.
2024-05-10 20:12:21,775 - INFO - Judgement 2 of 9019
2024-05-10 20:12:21,775 - INFO - 650c3a798fc6aa8c775aa20b
2024-05-10 20:12:21,775 - INFO - Is already analyzed.
2024-05-10 20:12:21,776 - INFO - Judgement 3 of 9019
2024-05-10 20:12:21,776 - INFO - 650c3a798fc6aa8c775aa20c
2024-05-10 20:12:21,776 - INFO - Is already analyzed.
2024-05-10 20:12:21,776 - INFO - Judgement 4 of 9019
2024-05-10 20:12:21,777 - INFO - 650c3a7a8fc6aa8c775aa20d
2024-05-10 20:12:21,777 - INFO - Is already analyzed.
2024-05-10 20:12:21,777 - INFO - Judgement 5 of 9019
2024-05-10 20:12:21,777 - INFO - 650c3a7a8fc6aa8c775aa20e
2024-05-10 20:12:21,778 - INFO - Is already analyzed.
2024-05-10 20:12:21,778 - INFO - Judgement 6 of 9019
2024-05-10 20:12:21,778 - INFO - 650c3a7a8fc6aa8c775aa20f
2024-05-10 20:12:21,778 - INFO - Is already analyzed.
2024-05-10